# Examining Racial Discrimination in the US Job Market

### Background
Racial discrimination continues to be pervasive in cultures throughout the world. Researchers examined the level of racial discrimination in the United States labor market by randomly assigning identical résumés to black-sounding or white-sounding names and observing the impact on requests for interviews from employers.

### Data
In the dataset provided, each row represents a resume. The 'race' column has two values, 'b' and 'w', indicating black-sounding and white-sounding. The column 'call' has two values, 1 and 0, indicating whether the resume received a call from employers or not.

Note that the 'b' and 'w' values in race are assigned randomly to the resumes when presented to the employer.

<div class="span5 alert alert-info">
### Exercises
You will perform a statistical analysis to establish whether race has a significant impact on the rate of callbacks for resumes.

Answer the following questions **in this notebook below and submit to your Github account**. 

   1. What test is appropriate for this problem? Does CLT apply?
   2. What are the null and alternate hypotheses?
   3. Compute margin of error, confidence interval, and p-value. Try using both the bootstrapping and the frequentist statistical approaches.
   4. Write a story describing the statistical significance in the context or the original problem.
   5. Does your analysis mean that race/name is the most important factor in callback success? Why or why not? If not, how would you amend your analysis?

You can include written notes in notebook cells using Markdown: 
   - In the control panel at the top, choose Cell > Cell Type > Markdown
   - Markdown syntax: http://nestacms.com/docs/creating-content/markdown-cheat-sheet


#### Resources
+ Experiment information and data source: http://www.povertyactionlab.org/evaluation/discrimination-job-market-united-states
+ Scipy statistical methods: http://docs.scipy.org/doc/scipy/reference/stats.html 
+ Markdown syntax: http://nestacms.com/docs/creating-content/markdown-cheat-sheet
+ Formulas for the Bernoulli distribution: https://en.wikipedia.org/wiki/Bernoulli_distribution
</div>
****

In [18]:
import pandas as pd
import numpy as np
from scipy import stats
from statsmodels.stats import proportion
import statsmodels.api as sm
import statsmodels

In [3]:


def ecdf(data):
    """Compute( ECDF for a one-dimensional array of measurements."""
    # Number of data points: n
    n=len(data)
    # x-data for the ECDF: x
    x = np.sort(data)

    # y-data for the ECDF: y
    y = np.arange(1, n+1) / n

    return x, y



def draw_bs_reps(data, func, size=1):
    """Draw bootstrap replicates."""

    # Initialize array of replicates: bs_replicates
    bs_replicates = np.empty(size)

    # Generate replicates
    for i in range(size):
        bs_replicates[i] = bootstrap_replicate_1d(data, func)

    return bs_replicates



def bootstrap_replicate_1d(data, func):
    return func(np.random.choice(data, size=len(data)))

def draw_perm_reps(data_1, data_2, func, size=1):
    """Generate multiple permutation replicates."""

    # Initialize array of replicates: perm_replicates
    perm_replicates = np.empty(size)

    for i in range(size):
        # Generate permutation sample
        perm_sample_1, perm_sample_2 = permutation_sample(data_1, data_2)

        # Compute the test statistic
        perm_replicates[i] = func(perm_sample_1, perm_sample_2)

    return perm_replicates

def permutation_sample(data1, data2):
    """Generate a permutation sample from two data sets."""

    # Concatenate the data sets: data
    data = np.concatenate((data1, data2))

    # Permute the concatenated array: permuted_data
    permuted_data = np.random.permutation(data)

    # Split the permuted array into two: perm_sample_1, perm_sample_2
    perm_sample_1 = permuted_data[:len(data1)]
    perm_sample_2 = permuted_data[len(data1):]

    return perm_sample_1, perm_sample_2

In [27]:
data = pd.io.stata.read_stata('data/us_job_market_discrimination.dta')

In [5]:
# number of callbacks for black-sounding names
print(sum(data[data.race=='b'].call),
sum(data[data.race=='w'].call))

157.0 235.0


In [6]:
data.head()

,id,ad,education,ofjobs,yearsexp,honors,volunteer,military,empholes,occupspecific,...,compreq,orgreq,manuf,transcom,bankreal,trade,busservice,othservice,missind,ownership
0,b,1,4,2,6,0,0,0,1,17,...,1.0,0.0,1.0,0.0,0.0,0.0,0.0,0.0,0.0,
1,b,1,3,3,6,0,1,1,0,316,...,1.0,0.0,1.0,0.0,0.0,0.0,0.0,0.0,0.0,
2,b,1,4,1,6,0,0,0,0,19,...,1.0,0.0,1.0,0.0,0.0,0.0,0.0,0.0,0.0,
3,b,1,3,4,6,0,1,0,1,313,...,1.0,0.0,1.0,0.0,0.0,0.0,0.0,0.0,0.0,
4,b,1,3,3,22,0,0,0,0,313,...,1.0,1.0,0.0,0.0,0.0,0.0,0.0,1.0,0.0,Nonprofit


<div class="span5 alert alert-success">
<p>Your answers to Q1 and Q2 here</p>
</div>

This can be approximated with a two sample z test since the sample size is so large so CLT does apply. 

The null hypothesis is that the proportion of resumes with black soudning names that got a call back is the same as the proportion of resumes with white sounding names are the same, p1 = p2, and the alternalte is that they are not the same, p1 != p2.

H0: p1 = p2

Ha: p1 != p2

In [7]:
w = data[data.race=='w']
b = data[data.race=='b']
print(len(w), len(b))

2435 2435


In [8]:
# Your solution to Q3 here

In [50]:
mean_call_white = len(w[w.call == 1])/len(w)
mean_call_black = len(b[b.call == 1])/len(b)
mean_call = len(data[data.call == 1])/len(data)
print(mean_call, mean_call_white, mean_call_black)
std_black = np.sqrt((mean_call_black*(1-mean_call_black))/len(b))
std_white = np.sqrt((mean_call_white*(1-mean_call_white))/len(w))
print(std_white, std_black)

0.08049281314168377 0.09650924024640657 0.06447638603696099
0.00598407217813 0.00497712144281


In [52]:
# Construct arrays of data: black, white
black = np.array([True] * 157 + [False] * 2278)
white = np.array([True] * 235 + [False] * 2200)

perm_replicates_bs = np.empty(10000)
perm_replicates_bs_w = np.empty(10000)
for i in range(10000):
    perm_replicates_bs[i] = draw_bs_reps(black, np.mean)
    perm_replicates_bs_w[i] = draw_bs_reps(white, np.mean)
    perm_replicates_diff = perm_replicates_bs_w -perm_replicates_bs
# Compute and print p-value: p
p = np.sum(perm_replicates_diff <= 0 ) / len(perm_replicates_diff)
print('p-value =', p)


print('the 95% confidence interval is : ' +str(np.percentile(perm_replicates_diff, [2.5, 97.5])))

p-value = 0.0
the 95% confidence interval is : [ 0.01683778  0.04722793]


In [56]:
mean_diff = mean_call_black-mean_call_white
std = np.sqrt(std_black**2 +std_white**2)
interval = [-1.96*std+abs(mean_diff),1.96*std+abs(mean_diff)]
print('the 95% confidence interval is: ' +str(interval))

z = mean_diff/std
print('z score = '+str(z))
print('p-value = ' +str(stats.norm.cdf(z)))

the 95% confidence interval is: [0.016777447859559147, 0.047288260559332024]
z score = -4.11555043573
p-value = 1.93128260376e-05


<div class="span5 alert alert-success">
<p> Your answers to Q4 and Q5 here </p>
</div>

# Question 4 + 5

Write a story describing the statistical significance in the context or the original problem.

In this case  the pvalue of ~ .00001 means that race almost certianly has an effect in the rate of callbacks for resume submissions. Our confidence interval shows that this effect translates into white people getting a callback likely within about 1.6% to 4.7% more than black people. 

That being said this does not mean that race is definetely the most important factor in predicting the likelyhood of a call back. I created a quick correlation matrix and found out that there were a few categories that were more correlated with getting a call back in this case but race is still one of the most highly correlated. Some regression testing could be done to tease out some more details. 

In [51]:
data.corr()['call'].sort_values()

race_b                -0.058872
fracdropout           -0.056671
lmedhhinc_empzip      -0.049879
req                   -0.041699
educreq               -0.033864
orgreq                -0.033416
fracwhite_empzip      -0.032989
branch_sales          -0.029126
computerskills        -0.028813
manuf                 -0.028785
workinschool          -0.027888
branch_emp            -0.026909
ownership_Nonprofit   -0.026264
l                     -0.025835
compreq               -0.024907
fracblack             -0.022130
trade                 -0.021853
salesrep              -0.021584
military              -0.020577
manager               -0.020269
expreq                -0.019250
ownership_Private     -0.014865
supervisor            -0.012061
bankreal              -0.008996
col                   -0.008479
missind               -0.007555
education             -0.005748
ofjobs                 0.002311
retailsales            0.002336
comreq                 0.002421
                         ...   
secretar